In [ ]:

import requests
import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from typing import Dict, List, Optional, Tuple
import time
import logging
from urllib.parse import urlparse, urljoin
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from datetime import datetime
from ratelimit import limits, sleep_and_retry

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('wordpress_checker.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Rate limits
CALLS_PER_MINUTE = 60
ONE_MINUTE = 60

@sleep_and_retry
@limits(calls=CALLS_PER_MINUTE, period=ONE_MINUTE)
def rate_limited_api_call(func):
    return func()

class WordPressIndexChecker:
    def __init__(self, gsc_credentials_file: str, domains: List[str], batch_size: int = 5):
        """Initialize the WordPress Index Checker."""
        self.domains = [self._format_domain_url(domain) for domain in domains]
        self.batch_size = batch_size
        self.credentials = self._load_credentials(gsc_credentials_file)
        self.service = build("searchconsole", "v1", credentials=self.credentials)
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })

    def _load_credentials(self, credentials_file: str) -> Credentials:
        """Load Google Search Console credentials."""
        try:
            return Credentials.from_service_account_file(
                credentials_file,
                scopes=["https://www.googleapis.com/auth/webmasters"]
            )
        except Exception as e:
            logger.error(f"Failed to load credentials: {e}")
            raise

    def _format_domain_url(self, domain: str) -> str:
        """Format domain URL with proper scheme."""
        domain = domain.strip().lower()
        if not domain.startswith(('http://', 'https://')):
            domain = 'https://' + domain
        return domain.rstrip('/')

    def _get_site_url(self, url: str) -> str:
        """Get site URL from any URL."""
        parsed = urlparse(url)
        return f"{parsed.scheme}://{parsed.netloc}/"

    def get_all_posts(self, domain: str, max_retries: int = 3) -> List[Dict]:
        """Get all posts from a WordPress domain."""
        posts = []
        page = 1
        retries = max_retries
        base_url = domain

        while True:
            try:
                endpoint = f"{base_url}/wp-json/wp/v2/posts"
                logger.info(f"Fetching page {page} from: {endpoint}")

                response = self.session.get(
                    endpoint,
                    params={
                        'page': page,
                        'per_page': 100,
                        'status': 'publish'
                    },
                    timeout=15
                )
                response.raise_for_status()

                current_posts = response.json()
                if not current_posts:
                    break

                posts.extend(current_posts)
                page += 1
                retries = max_retries  # Reset retries on success
                time.sleep(1)  # Rate limiting

            except requests.exceptions.HTTPError as e:
                if response.status_code == 400:
                    if response.json().get('code') == 'rest_post_invalid_page_number':
                        logger.info(f"No more pages available for {base_url} (page {page}).")
                        break
                    else:
                        logger.error(f"400 Bad Request error encountered for {base_url} (page {page}). Response: {response.text}. Skipping to next domain.")
                        break
                else:
                    logger.error(f"Error fetching posts from {base_url} (page {page}, attempt {max_retries - retries + 1}): {e}. Response: {response.text}")
                    retries -= 1
                    if retries <= 0:
                        logger.error(f"Max retries reached for {base_url}")
                        break
                    time.sleep(2 ** (max_retries - retries))  # Exponential backoff
            except requests.exceptions.RequestException as e:
                logger.error(f"Error fetching posts from {base_url} (page {page}, attempt {max_retries - retries + 1}): {e}")
                retries -= 1
                if retries <= 0:
                    logger.error(f"Max retries reached for {base_url}")
                    break
                time.sleep(2 ** (max_retries - retries))  # Exponential backoff

        return posts

    def check_google_indexing(self, url: str) -> Optional[bool]:
        """Check if a URL is indexed in Google Search."""
        try:
            url = self._format_domain_url(url)
            site_url = self._get_site_url(url)
            
            request = self.service.urlInspection().index().inspect(
                body={
                    "inspectionUrl": url,
                    "siteUrl": site_url
                }
            )
            response = request.execute()
            
            inspection_result = response.get("inspectionResult", {})
            index_status = inspection_result.get("indexStatusResult", {})
            coverage_state = index_status.get("coverageState")
            
            return coverage_state == "INDEXED"
        except Exception as e:
            logger.error(f"Error checking indexing status for {url}: {e}")
            return None

    def process_single_post(self, domain: str, post: Dict) -> Optional[Dict]:
        """Process a single WordPress post."""
        try:
            post_title = post.get("title", {}).get("rendered", "No Title")
            post_url = post.get("link", "No URL")
            post_date = post.get("date", "No Date")
            
            def check_index():
                return self.check_google_indexing(post_url)
            
            indexed = rate_limited_api_call(check_index)
            
            days_not_indexed = None
            if indexed is False and post_date != "No Date":
                post_publish_date = datetime.strptime(post_date, "%Y-%m-%dT%H:%M:%S")
                days_not_indexed = (datetime.now() - post_publish_date).days
            
            return {
                "Domain": domain,
                "Post Title": post_title,
                "Post URL": post_url,
                "Post Date": post_date,
                "Indexed": "Yes" if indexed else "No" if indexed is False else "Error",
                "Days Not Indexed": days_not_indexed if days_not_indexed is not None else "N/A",
                "Check Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
        except Exception as e:
            logger.error(f"Error processing post {post.get('link', 'Unknown URL')}: {e}")
            return None

    def process_single_domain(self, domain: str) -> List[Dict]:
        """Process all posts from a single domain."""
        results = []
        logger.info(f"Processing domain: {domain}")
        
        posts = self.get_all_posts(domain)
        if not posts:
            logger.warning(f"No posts found for domain: {domain}")
            return results
            
        for post in posts:
            try:
                post_data = self.process_single_post(domain, post)
                if post_data:
                    results.append(post_data)
            except Exception as e:
                logger.error(f"Error processing post in domain {domain}: {e}")
                continue
                
            time.sleep(1)  # Rate limiting between posts
            
        return results

    def process_domains_in_batches(self) -> pd.DataFrame:
        """Process all domains in batches."""
        all_results = []
        total_batches = (len(self.domains) + self.batch_size - 1) // self.batch_size
        
        for i in range(0, len(self.domains), self.batch_size):
            batch = self.domains[i:i + self.batch_size]
            current_batch = i // self.batch_size + 1
            logger.info(f"Processing batch {current_batch} of {total_batches}")
            
            for domain in batch:
                try:
                    results = self.process_single_domain(domain)
                    all_results.extend(results)
                except Exception as e:
                    logger.error(f"Error processing domain {domain}: {e}")
                
                time.sleep(2)  # Delay between domains
            
            if current_batch < total_batches:
                logger.info("Waiting between batches...")
                time.sleep(10)  # Delay between batches
        
        return pd.DataFrame(all_results)

    def save_excel_report(self, df: pd.DataFrame, output_file: str):
        """Save results to a formatted Excel report."""
        if df.empty:
            logger.warning("No data to save to Excel report")
            return

        try:
            writer = pd.ExcelWriter(output_file, engine='openpyxl')
            
            for domain in df['Domain'].unique():
                domain_data = df[df['Domain'] == domain]
                sheet_name = urlparse(domain).netloc[:31]  # Excel sheet name length limit
                
                domain_data.to_excel(writer, sheet_name=sheet_name, index=False)
                worksheet = writer.sheets[sheet_name]
                
                # Styling
                header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
                header_font = Font(color='FFFFFF', bold=True)
                border = Border(
                    left=Side(style='thin'), 
                    right=Side(style='thin'),
                    top=Side(style='thin'), 
                    bottom=Side(style='thin')
                )
                
                # Format headers
                for col in range(1, len(domain_data.columns) + 1):
                    cell = worksheet.cell(row=1, column=col)
                    cell.fill = header_fill
                    cell.font = header_font
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                    cell.border = border
                    
                    # Auto-adjust column width
                    column_letter = get_column_letter(col)
                    max_length = max(
                        len(str(domain_data.columns[col-1])),
                        domain_data[domain_data.columns[col-1]].astype(str).map(len).max()
                    )
                    adjusted_width = min(max_length + 2, 50)  # Cap width at 50
                    worksheet.column_dimensions[column_letter].width = adjusted_width
                
                # Format data cells
                for row in range(2, len(domain_data) + 2):
                    for col in range(1, len(domain_data.columns) + 1):
                        cell = worksheet.cell(row=row, column=col)
                        cell.border = border
                        cell.alignment = Alignment(horizontal='left', vertical='center')
                        
                        # Color coding for Indexed column
                        if domain_data.columns[col-1] == 'Indexed':
                            value = cell.value
                            if value == 'Yes':
                                cell.fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
                            elif value == 'No':
                                cell.fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
                            elif value == 'Error':
                                cell.fill = PatternFill(start_color='FFEB9C', end_color='FFEB9C', fill_type='solid')
            
            writer.close()
            logger.info(f"Excel report saved successfully to {output_file}")
            
        except Exception as e:
            logger.error(f"Failed to save Excel report: {e}")
            raise

def main():
    GSC_CREDENTIALS_FILE = "/content/wordpress-index-checker-448410-4533712f0b6d.json"
    DOMAINS = [
        "tclottery-app.in",

    ]
    
    try:
        # Initialize checker with batch processing
        checker = WordPressIndexChecker(GSC_CREDENTIALS_FILE, DOMAINS, batch_size=5)
        
        # Process all domains and get results
        results_df = checker.process_domains_in_batches()
        
        if not results_df.empty:
            # Generate output filename with timestamp
            output_file = f"{DOMAINS}_wordpress_indexing_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
            
            # Save the report
            checker.save_excel_report(results_df, output_file)
            logger.info(f"Report generated successfully: {output_file}")
        else:
            logger.warning("No data was collected. Please check if the domains are accessible.")
        
    except Exception as e:
        logger.error(f"Script execution failed: {e}")
        raise

if __name__ == "__main__":
    main()